In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 1. Implement Scaled Dot Product Attention Mechanism

$$ \text{SoftMax} \left( \frac{Q}{T_e}K^T\right) V, \quad T_e = \sqrt{d_k}$$

In [2]:
class ScaledDotProductAttention(nn.Module):
    """ Scaled Dot Product Attention """
    def __init__(self, temperature, attn_dropout = 0.1):
        super(ScaledDotProductAttention).__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)

    def forward(self, q, k, v, mask = None):
        attn = torch.matmul(q/self.temperature, k.transpose(2, 3))

        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)

        attn = self.dropout(F.softmax(attn, dim = -1))
        output = torch.matmul(attn, v)

        return output, attn

# 2. Implement Multi-Head Attention

### Run several scaled dot product attention, each mapping to a different feature map, meaning each has its own Q, K, and V matrices.

- $ d_{model} \rightarrow $ dimensionality of the representation of each token. In other words it is the number of values that are associated with a word. [Embedding Space]

- $ d_k \rightarrow $ feature space of reduced dimensionality. It represents an extraction of some sort of information from the complete dimensionality  $d_{model}$

- $ d_v \rightarrow w_V $ represents the matrix that when multiplied by the embedding ($d_{model}$) return the value vector. The value vector is what must be added to the original embedding to provide context



$$ {W_Q}_{\text{Multi-head}} =
\left(
  \begin{array}{c|c|c|c}
      &     &        &             \\
  {W_Q}_1 & {W_Q}_2 & \cdots & {W_Q}_{n_{Heads}} \\
      &     &        &  
\end{array} \right), \quad

{W_K}_{\text{Multi-head}} =
\left(
  \begin{array}{c|c|c|c}
      &     &        &             \\
  {W_K}_1 & {W_K}_2 & \cdots & {W_K}_{n_{Heads}} \\
      &     &        &  
\end{array} \right), \quad

{W_V}_{\text{Multi-head}} =
\left(
  \begin{array}{c|c|c|c}
      &     &        &             \\
  {W_V}_1 & {W_V}_2 & \cdots & {W_V}_{n_{Heads}} \\
      &     &        &  
\end{array} \right)
$$

$${W_Q}_x, {W_K}_x \quad \text{have dimensions} \quad (d_k \times d_{model})$$
$${W_V}_x \quad \text{has dimensions} \quad (d_{model} \times d_{model})$$

This way $W_V$ has many more paramenters than $W_Q$ and $W_K$ so it is often split into two matrices ${W_V}_{\uparrow}$, and ${W_V}_{\downarrow}$

Then, instead of having:
$$ {W_V}_1 \times E_{token}, \quad \text{we write} \quad {W_V}_{1\uparrow} \times {W_V}_{1\downarrow} \times E_{token}$$

Here ${W_V}_{\uparrow}$ has dimensions $(d_{model} \times d_v)$, and ${W_V}_{\downarrow}$ has dimensions $(d_v \times d_{model})$

<sub>*notation for matrix dimensions is (rows x columns). Here we are considering $W \times E_\uparrow$ but nn.Linear() is actually more similar to $E_\rightarrow \times W^T$</sub>

In [162]:
class MultiHeadAttention(nn.Module):
    """ Multi-Head Attention """
    def __init__(self, n_head, d_model, d_k, d_v, dropout = 0.1):
        super(MultiHeadAttention).__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v
        
        # Linear Transformations: y = x @ A^T + b
        # Where the values inside A are trainable

        # d_model - embedding size
        # n_head - number of attention heads
        # d_k - reduced feature space

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias = False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias = False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias = False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias = False)

        # Initialize attention
        self.attention = ScaledDotProductAttention(temperature = d_k ** 0.5)

        # Dropout randomly deactivates neuron heads. This is used as a regularization
        # technique and is effective in reducing overfitting.
        self.dropout = nn.Dropout(dropout)

        # Standard Scaler Normalization initialized.
        # Eps is a value added to denominator and is used for increased stability
        self.layer_norm = nn.LayerNorm(d_model, eps = 1e-6)

    def forward(self, q, k, v, mask = None):
        """
        q, k, v in the encoder layer is just the encoddings
        
        sz_b -> size of the batch being processed.
        len_q -> size of the phrase
        len_k -> size of the phrase
        v -> size of the phrase

        Refer to Extra Material (a.) to understand the logic behind
        this calculations
        """

        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)

        # Retain the input to be added in front (residual connection)
        residual = q

        # Obtain the Query, Key and Value matrices for all sentences in the batch
        # and for all the attention heads
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        # Transpose values to calculate self attention: batch x nheads x nwords x reduced_embedd
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)

        if mask is not None:
            mask = mask.unsqueeze(1) # Head axis broadcasting

        
        # Perform the Scaled Dot Product Attention Calculation
        q, attn = self.attention(q, k, v, mask = mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine last two dimensions to concatenate all the heads together: b x lq x (n*dv)

        # Return to the previous dimension order so we can include the residual connection
        # We must maintain this order for consequent training reinjection
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        # Perform layer normalization to each attention head ??? (revisit)
        q = self.layer_norm(q)

        # Attn is the calculation attention value (argument of the softmax)
        # q if the output of the multihead attention layer.
        return q, attn 

## 2.1 Multilayer Perceptron after the multi-head attention

In [4]:
class PositionwiseFeedForward(nn.Module):
    """ Two-feed-forward-layer module """

    def __init__(self, d_in, d_hid, dropout = 0.1):
        super(PositionwiseFeedForward).__init__()
        self.w_1 = nn.Linear(d_in, d_hid) # position-wise
        self.w_2 = nn.Linear(d_hid, d_in) # position-wise
        self.layer_norm = nn.LayerNorm(d_in, eps = 1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual

        x = self.layer_norm(x)

        return x

# 3. Encoder/Decoder Implementations

## 3.1 Encoder

In [7]:
class EncoderLayer(nn.Module):
    """ Composed of Two Layers """
    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout = 0.1):
        super(EncoderLayer).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout)

    def forward(self, enc_input, slf_attn_mask = None):
        enc_output, enc_slf_attn = self.slf_attn(enc_input, enc_input, enc_input, mask = slf_attn_mask)
        enc_output = self.pos_ffn(enc_output)

        return enc_output, enc_slf_attn

## 3.2 Decoder

In [8]:
class DecoderLayer(nn.Module):
    """ Composed of Three Layers """
    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout = 0.1):
        super(DecoderLayer).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout)
        self.enc_layer = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout)

    def forward(self, dec_input, d_inner, enc_output, slf_attn_mask = None, dec_enc_attn_mask = None):
        dec_output, dec_slf_attn = self.slf_attn(dec_input, dec_input, dec_input, slf_attn_mask)
        dec_output, dec_enc_attn = self.enc_attn(dec_output, enc_output, enc_output, dec_enc_attn_mask)
        dec_output = self.pos_ffn(dec_output)
        return dec_output, dec_slf_attn, dec_enc_attn

# ! Extra Material

## a. understand Query, Key and Value matrices

In [158]:
d_model = 12000
n_head = 6 
d_k = 128
b_size = 3
max_phrase_size = 5
test_l = nn.Linear(d_model, d_k * n_head, bias = False)
test = test_l.get_parameter('weight')
embedd = torch.randn(b_size, max_phrase_size, d_model)

In [159]:
test.size()

torch.Size([768, 12000])

In [160]:
test_l(embedd).view(b_size, max_phrase_size, n_head, d_k).size()

torch.Size([3, 5, 6, 128])

#### <span style="color:red">NOTE:</span> After this calculation we just transpose dimensions 1 and 2 and obtain the Q, K and V matrices for 3 batches, each having 6 attention heads for phrases of length 5 where each word has and embedding size of 128

## b. understand word Embedding

In [117]:
n_src_vocab = 5000
d_word_vec = 128
src_word_emb = nn.Embedding(n_src_vocab, d_word_vec)

In [146]:
vocab = {"cake": 0, "there": 1, "hey": 2, "is": 3, "good": 4999}
word_indexes = [vocab[w] for w in ["cake", "is", "good"]]
word_indexes

[0, 3, 4999]

In [147]:
word_vectors = src_word_emb(torch.tensor(word_indexes))
word_vectors.size()

torch.Size([3, 128])